# Lorenz system: manifold plots

In [1]:
%matplotlib inline

from tqdm import tqdm
import sys
import numpy as np
import matplotlib
import torch
import logging
from matplotlib import pyplot as plt, cm
from matplotlib.collections import PolyCollection
from matplotlib.tri import Triangulation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree

sys.path.append("../../")
from experiments.datasets import LorenzSimulator
from experiments.architectures.vector_transforms import create_vector_transform
from manifold_flow.flows import ManifoldFlow
import plot_settings as ps

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.DEBUG,
)
for key in logging.Logger.manager.loggerDict:
    if "experiments" not in key and "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [2]:
# ps.setup()

## Set up simulator

In [ ]:
sim = LorenzSimulator()


11:19 experiments.datasets.lorenz    INFO    Solving Lorenz system with parameters sigma = 10.0, beta = 2.6666666666666665, rho = 28.0, initial conditions x0 = [1. 1. 1.], for 10000000 time steps from 0 to 100.0


## Load flows

In [4]:
def load_model(filename):
    outer_transform = create_vector_transform(
        3,
        5,
        linear_transform_type="permutation",
        base_transform_type="rq-coupling",
        context_features=None,
        dropout_probability=0.,
        tail_bound=3.,
        num_bins=5,
    )
    inner_transform = create_vector_transform(
        2,
        5,
        linear_transform_type="permutation",
        base_transform_type="rq-coupling",
        context_features=sim.parameter_dim(),
        dropout_probability=0.,
        tail_bound=3.,
        num_bins=5,
    )

    model = ManifoldFlow(
        data_dim=3,
        latent_dim=2,
        outer_transform=outer_transform,
        inner_transform=inner_transform,
        apply_context_to_outer=False,
    )
    
    model.load_state_dict(
        torch.load("../data/models/{}.pt".format(filename), map_location=torch.device("cpu"))
    )
    _ = model.eval()
    
    return model

In [5]:
mf = load_model("mf_2_lorenz_april")


11:19 manifold_flow.transforms.proje DEBUG   Set up projection from vector with dimension 3 to vector with dimension 2
11:19 manifold_flow.flows.base       DEBUG   Flow has 0.4 M parameters (0.4 M trainable) with an estimated size of 1.7 MB
11:19 manifold_flow.flows.manifold_f DEBUG     Outer transform: 0.2 M parameters
11:19 manifold_flow.flows.manifold_f DEBUG     Inner transform: 0.2 M parameters


## Find points for evaluation: start with latent grid, triangulate, add points in x space where necessary

In [32]:
def make_grid(boundary, res):
    grid_each = np.linspace(-boundary, boundary, res)
    x, y = np.meshgrid(grid_each, grid_each)
    xy = np.vstack((x.flatten(), y.flatten())).T
    return xy


def decode(zs, batchsize = 1000):
    xs = []
    n_batches = (len(zs) - 1) // batchsize + 1
    for i in range(n_batches):
        print("z batch {} / {}".format(i + 1, n_batches))
        z_batch = zs[i*batchsize:(i+1)*batchsize]
        z_ = torch.tensor(z_batch, dtype=torch.float)
        x_ = mf.decode(u=z_)
        xs += list(x_.detach().numpy().flatten())

    xs = np.array(xs).reshape((-1, 3))
    return xs


def triangle_area(a, b, c):
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ca = np.linalg.norm(c - a)
    s = (ab + bc + ca) / 2.
    return (s * (s-ab) * (s-bc) * (s-ca))**0.5
    
    
def uniform_points_in_triangle(a, b, c, n=1):
    p = np.random.uniform(size=n)[:,np.newaxis]
    q = np.random.uniform(size=n)[:,np.newaxis]
    p_ = np.where(p + q > 1., 1. - p, p)
    q_ = np.where(p + q > 1., 1. - q, q)
    points = a[np.newaxis, :] + (b - a)[np.newaxis, :] * p_ + (c - a)[np.newaxis, :] * q_
    
    return points


In [33]:
z_seeds = make_grid(3., 100)
x_seeds = decode(z_seeds)
z_seeds.shape, x_seeds.shape


z batch 1 / 10
z batch 2 / 10
z batch 3 / 10
z batch 4 / 10
z batch 5 / 10
z batch 6 / 10
z batch 7 / 10
z batch 8 / 10
z batch 9 / 10
z batch 10 / 10


((10000, 2), (10000, 3))

In [34]:
goal_area = 0.01
n_additional_max = 100
triangulation = Triangulation(z_seeds[:,0], z_seeds[:,1])

z_mf = np.copy(z_seeds)

for t in triangulation.triangles:
    area = triangle_area(x_seeds[t[0]], x_seeds[t[1]], x_seeds[t[2]])
    n_additional = min(max(int(round(area / goal_area, 0)) - 1, 0), n_additional_max)
    
    if n_additional > 0:
        z_additional = uniform_points_in_triangle(
            z_seeds[t[0]],
            z_seeds[t[1]],
            z_seeds[t[2]],
            n=n_additional
        )
        z_mf = np.concatenate((z_mf, z_additional), axis=0)
        
z_mf.shape


(23940, 2)

In [35]:
x_mf = decode(z_mf)
z_mf.shape, x_mf.shape


z batch 1 / 24
z batch 2 / 24
z batch 3 / 24
z batch 4 / 24
z batch 5 / 24
z batch 6 / 24
z batch 7 / 24
z batch 8 / 24
z batch 9 / 24
z batch 10 / 24
z batch 11 / 24
z batch 12 / 24
z batch 13 / 24
z batch 14 / 24
z batch 15 / 24
z batch 16 / 24
z batch 17 / 24
z batch 18 / 24
z batch 19 / 24
z batch 20 / 24
z batch 21 / 24
z batch 22 / 24
z batch 23 / 24
z batch 24 / 24


((23940, 2), (23940, 3))

In [36]:
max_distance = 0.05

tree = cKDTree(x_seeds)
neighbors = tree.query_pairs(r=max_distance)
duplicates = [n[1] for n in neighbors]

x_mf = np.array([x for i, x in enumerate(x_mf) if i not in duplicates])
z_mf = np.array([z for i, z in enumerate(z_mf) if i not in duplicates])

x_mf.shape, z_mf.shape


((17727, 3), (17727, 2))

## Evaluate log likelihood

In [ ]:
batchsize = 400

logps = []

n_batches = (len(z_seeds) - 1) // batchsize + 1
for i in range(n_batches):
    print("Batch {} / {}".format(i + 1, n_batches))
    x_batch = x_seeds[i*batchsize:(i+1)*batchsize]
    x_batch_ = torch.tensor(x_batch, dtype=torch.float)
    _, logp_, _ = mf(x_batch_, mode="mf")
    logps.append(logp_.detach().numpy().flatten())

logp_mf = np.hstack(logps)


Batch 1 / 25
Batch 2 / 25
Batch 3 / 25
Batch 4 / 25
Batch 5 / 25
Batch 6 / 25
Batch 7 / 25
Batch 8 / 25
Batch 9 / 25
Batch 10 / 25
Batch 11 / 25


In [ ]:
x_traj = sim.trajectory()

## Triangulation

In [106]:
# Strongly based on https://discourse.matplotlib.org/t/trisurf-plots-with-independent-color-data/19033/2

min_area = 0.1
n_additional_max = 100
triangulation = Triangulation(z_seeds[:,0], z_seeds[:,1])

z_tri = []

for t in triangulation.triangles:
    area = triangle_area(x_mf[t[0]], x_mf[t[1]], x_mf[t[2]])
    n_additional = min(max(int(round(area / goal_area, 0)) - 1, 0), n_additional_max)
    
    if n_additional > 0:
        x_additional = uniform_points_in_triangle(
            x_seeds[t[0]],
            x_seeds[t[1]],
            x_seeds[t[2]],
            n=n_additional
        )
        x_seeds = np.concatenate((x_seeds, x_additional), axis=0)
        
x_seeds.shape






circ_max = 1000
triangulation = Triangulation(z_mf[:,0], z_mf[:,1])

x_tri = []
logp_tri = []
z_tri = []

for t in triangulation.triangles:
    circ = np.linalg.norm(x_mf[t[0]] - x_mf[t[1]]) + np.linalg.norm(x_mf[t[1]] - x_mf[t[2]]) + np.linalg.norm(x_mf[t[2]] - x_mf[t[0]])
    if circ > circ_max:
        continue
        
    z_tri.append(
        [
            [z_mf[t[0], 0], z_mf[t[0], 1]],
            [z_mf[t[1], 0], z_mf[t[1], 1]],
            [z_mf[t[2], 0], z_mf[t[2], 1]]
        ]
    )
    x_tri.append(
        [
            [x_mf[t[0], 0], x_mf[t[0], 1], x_mf[t[0], 2]],
            [x_mf[t[1], 0], x_mf[t[1], 1], x_mf[t[1], 2]],
            [x_mf[t[2], 0], x_mf[t[2], 1], x_mf[t[2], 2]]
        ]
    )
    logp_tri.append(
        (logp_mf[t[0]] + logp_mf[t[1]] + logp_mf[t[2]]) / 3.
    )

z_tri = np.array(z_tri)
x_tri = np.array(x_tri)
logp_tri = np.array(logp_tri)

logp_tri.shape, x_tri.shape, z_tri.shape


((19693,), (19693, 3, 3), (19693, 3, 2))

In [110]:
ccutoff, cmin, cmax = None, -7.5, -2.
x_boundary = 4.

def cmap(vals):
    return ps.CMAP(np.clip((vals - cmin) / (cmax - cmin), 0., 1.))


if ccutoff is None:
    x_tri_ = x_tri
    colors_tri_ = cmap(logp_tri)
    z_tri_ = z_tri
else:
    x_tri_ = x_tri[logp_tri > ccutoff]
    colors_tri_ = cmap(logp_tri[logp_tri > ccutoff])
    z_tri_ = z_tri[logp_tri > ccutoff]


## 2D plot

In [111]:
fig = plt.figure(figsize=(4,4))
ax = plt.gca()

coll = PolyCollection(z_tri_, facecolors=colors_tri_, edgecolors='')
ax.add_collection(coll)

plt.xlabel("$u_1$")
plt.ylabel("$u_2$")

plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.tight_layout()
plt.show()


## 3D plot

In [112]:
# rgb = cmap(logp_mf).reshape((res, res, 4))
# ls = LightSource(azdeg=90, altdeg=75)
# rgb_shaded = ls.shade_rgb(rgb[:,:,:3], elevation=x_grid[:,2].reshape((res, res)), fraction=1.0, vert_exag=0.5, blend_mode="soft")


In [113]:
fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
    
coll = Poly3DCollection(x_tri_, facecolors=colors_tri_, edgecolors='')
ax.add_collection(coll)

ax.plot(
    x_traj[:, 0],
    x_traj[:, 1],
    x_traj[:, 2],
    c="black",
    lw=0.5
)

ax.set_xlim3d(-x_boundary, x_boundary)
ax.set_ylim3d(-x_boundary, x_boundary)
ax.set_zlim3d(-x_boundary, x_boundary)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.set_xticklabels([""]*3)
ax.set_yticklabels([""]*3)
ax.set_zticklabels([""]*3)

# ax.view_init(50, 65)

plt.show()
# plt.savefig("../figures/lorenz_manifold.pdf")


In [115]:
fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
    
ax.scatter(
    x_mf[:, 0],
    x_mf[:, 1],
    x_mf[:, 2],
    c=cmap(logp_mf),
    s=5.
)

ax.plot(
    x_traj[:, 0],
    x_traj[:, 1],
    x_traj[:, 2],
    c="black",
    lw=0.5
)

ax.set_xlim3d(-x_boundary, x_boundary)
ax.set_ylim3d(-x_boundary, x_boundary)
ax.set_zlim3d(-x_boundary, x_boundary)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.set_xticklabels([""]*3)
ax.set_yticklabels([""]*3)
ax.set_zticklabels([""]*3)

# ax.view_init(50, 65)

plt.show()
# plt.savefig("../figures/lorenz_manifold.pdf")

